# Model Pipeline

`roboflow` : 객체 탐지 -> `predict2crop()` : 이미지 크롭 -> `CLOVA_api()`: OCR_api -> 텍스트 추출

- packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as plt
import requests
import base64
import uuid
import json
import time
import cv2
import requests
import platform
from google.colab.patches import cv2_imshow
import numpy as np
import os

## 데이터 경로지정

> 해당 파일은 다량의 이미지를 Object Detection 모델과 OCR 모델에 넣어봄으로 오류가 없는지 테스트하는 용도입니다.

- `image_folder` : 데이터가 있는 폴더 경로

In [2]:
image_folder = '/content/drive/MyDrive/Datas/train/' # 데이터가 존재하는 폴더 경로
files = os.listdir(image_folder)
image_files = [file for file in files] # 데이터 접근 : 다중 파일로 테스트 시, 파일 순회

## Modules

> 이미지에 대한 입력 -> 객체 탐지 -> OCR -> 텍스트 전처리

- 해당 과정에 필요한 모듈들 입니다.

In [9]:
def CLOVA_api(secret_key, api_url, image : np.array):
    """
    Usage : CLOVA api 호출

    Parameters
    ----------
    secret_key : api key
    api_url : api_url
    image : 크롭된 image

    Returns
    -------

    response : api 호출 결과 (ex. 200,400,404, ...)

    """
    # Convert np.ndarray to bytes
    if image is not None:
        _, buffer = cv2.imencode('.jpg', image)
        file_data = buffer.tobytes()

    request_json = {
        'images': [
            {
                'format': 'jpg',
                'name': 'demo',
                'data': base64.b64encode(file_data).decode()
                #'url': image_url
            }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000))
    }
    payload = json.dumps(request_json).encode('UTF-8')
    headers = {
      'X-OCR-SECRET': secret_key,
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", api_url, headers=headers, data = payload)

    return response


def imageOCR(response, img : np.array):
    """
    Usage : api 적용 결과 (OCR 결과) 시각화

    Parameters
    ----------
    response : api 호출 결과
    img : 크롭된 이미지

    Returns
    -------
    temp : 추출된 text
    image : 크롭된 이미지 내 OCR 적용 결과

    """
    try : result = response.json()
    except :
        print(response, 'AttributeError: Responsed \'int\' object' )

    with open('result.json', 'w', encoding='utf-8') as make_file:
        json.dump(result, make_file, indent="\t", ensure_ascii=False)

    # Error 처리
    try : print(result['images'][0]['message'])
    except : return None, None

    # respone.json()에서 띄어쓰기 처리
    texts = connectWord(result)

    # 이미지 시각화
    bBs = [b['boundingPoly'] for b in result['images'][0]['fields']]

    # bounding box 표시
    for box in bBs:
        vertices = np.array([(int(point['x']), int(point['y'])) for point in box['vertices']], np.int32)
        vertices = vertices.reshape((-1, 1, 2))
        img = cv2.polylines(img, [vertices], isClosed=True, color=(255, 0, 0), thickness=2)

    # 이미지 보여주기
    # cv2_imshow(img) # only colab
    # print(text)

    return texts, img

def connectWord(ocr_json):
    detected_texts = ''

    # 필드 정보 추출
    fields = ocr_json['images'][0].get('fields', [])

    # 각 필드에 대해 y좌표 계산
    word_list = []
    for field in fields:
        bounding_poly = field.get('boundingPoly')
        infer_text = field.get('inferText')

        if bounding_poly and infer_text:
            vertices = bounding_poly['vertices']
            y_coord = vertices[0]['y']  # 첫 번째 꼭짓점의 y좌표를 사용
            word = infer_text
            word_list.append({'word': word, 'y_coord': y_coord})

    # y좌표가 유사한 단어를 그룹화
    grouped_words = []
    if word_list:  # word_list가 비어있지 않은 경우에만 처리
        current_group = [word_list[0]]

        for i in range(1, len(word_list)):
            if abs(word_list[i]['y_coord'] - word_list[i-1]['y_coord']) < 10:
                current_group.append(word_list[i])
            else:
                grouped_words.append(current_group)
                current_group = [word_list[i]]

        if current_group:
            grouped_words.append(current_group)

    # 결과 출력
    # print(f'File: {file_name}')
    for group in grouped_words:
        # 그룹 내의 단어들을 하나의 문자열로 합침
        group_text = ' '.join([word_info['word'] for word_info in group])

        # 문자열이 정수로만 이루어져 있지 않은 경우에만 출력
        if not group_text.isdigit():
            detected_texts += group_text

    return detected_texts

def modelPredict(model, input_Data):
    try : predictions_data = model.predict(input_Data, confidence=50, overlap=50).json()
    except :
        print('Predict Error')
        return None
    # image_path = predictions_data['predictions'][0]['image_path']  # Assuming all predictions have the same image path

    return predictions_data

def predict2crop(model, folder_path, image_file, resize = 256):
    """
    Usage : 객체 탐지 및 원본 및 크롭 이미지 return

    Parameters
    ----------
    model : Object Detection Model
    folder_path : 데이터 폴더 경로
    image_file : 데이터 파일 경로

    Returns : (original_image, cropped_image)
    -------
    type : tuple
    original_image : 원본 이미지
    cropped_image : 크롭된 이미지

    """
    print("="*50)
    print(image_file)
    image_path = folder_path + image_file
    img_size = resize # img_size * img_size

    org_img = cv2.imread(image_path)
    if org_img is None:
        print(f"Error: Unable to read the image file {image_path}")
        return None, None

    rsz_img = cv2.resize(org_img, (img_size, img_size), interpolation= cv2.INTER_AREA)
    adc_img = auto_adjust_contrast(rsz_img)


    predictions_data = modelPredict(model, input_Data = adc_img) # resize된 img속에서 찾은 bbox 좌표

    print('Detected : ', len(predictions_data['predictions']))

    if predictions_data is None :
        print('Detect Nothing. It\'s Too Close')
        return org_img, org_img

    if len(predictions_data['predictions']) > 1:
        print('! Error : Multiple detection Error. Take more closer')
        return None, None

    orgBBcoor = predBBcoor(org_img, predictions_data)
    cropped_img = imgCrop(org_img, orgBBcoor) #원본이미지에서 crop

    return org_img, cropped_img

def imgCrop(img, bbCoor):
    """
    Usage : resize된 이미지에서의 바운딩 박스 좌표를 원본 이미지의 바운딩 박스 좌표로 변환

    Parameters
    ----------
    img : 원본 이미지(no resize)
    bbCoor : (x, y, width, height) -> 원본 이미지에서의 bbox 좌표

    Returns : np.array
    -------
    cropped_img

    """
    if None in bbCoor : return img

    x, y, width, height = bbCoor # real coordinate

    half_w, half_h = round(width/2), round(height/2)

    cropped_img = img[abs(y-half_h) : y+half_h, abs(x-half_w ): x+half_w] # img crop

    # # 결과 시각화
    # cv2.rectangle(img, (x - half_w, y - half_h), (x + half_w, y + half_h), (0, 255, 0), 2)
    # print('\n Original')
    # cv2_imshow(img) # only colab
    # print('\n Cropped')
    # cv2_imshow(cropped_img) # only colab

    return cropped_img

def predBBcoor(org_img : np.array, pred_data : list, resize=256):
    """
    Usage : resize된 이미지에서의 바운딩 박스 좌표를 원본 이미지의 바운딩 박스 좌표로 변환

    Parameters
    ----------
    org_img : 원본 이미지(no resize)
    pred_data : list

    Returns : tuple
    -------
    x, y, width, height

    """
    try : prediction = pred_data['predictions'][0] # dict
    except : return None, None, None, None # 탐지 되지 않을 시

    # resized bbox coordinate
    rx, ry, rwidth, rheight = int(prediction['x']), int(prediction['y']), int(prediction['width']), int(prediction['height'])
    half_w, half_h = round(rwidth/2), round(rheight/2)

    # real h, w
    h, w, _ = org_img.shape
    # real bbox coordinate
    x, y, width, height = round((w*rx)/resize), round((h*ry)/resize), round((rwidth*w)/256), round((rheight*h)/256)

    return x, y, width, height

# preprocessing : auto_adjust_contrast
def auto_adjust_contrast(image : np.array):
    # Flatten the image to 1D array
    try : flat_image = image.flatten()
    except : return None

    # Compute the histogram
    histogram, bins = np.histogram(flat_image, bins=256, range=(0, 256))

    # Compute the cumulative distribution function (CDF)
    cdf = histogram.cumsum()

    # Normalize the CDF
    cdf_normalized = cdf / cdf.max()

    # Perform histogram equalization
    equalized_image = np.interp(flat_image, bins[:-1], cdf_normalized * 255).reshape(image.shape)

    return equalized_image.astype(np.uint8) # np.array

- model load

In [4]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="eyKD4VJQ4nRqtosRytMg")
project = rf.workspace().project("price-tag-dxlmv")
model = project.version(15).model

model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 

loading Roboflow workspace...
loading Roboflow project...


In [15]:
image = input_img
# OCR api
secret_key = "Y0l6ZHF1Um9CSWp3aHpJU3JDeFdpUGp1cG16T3hFQkg="
api_url = 'https://p0fsnflvaw.apigw.ntruss.com/custom/v1/27259/8a921c4c7d4e552c974b102e64c6227f3a2995ca938c066ddeb1442d6bf4b67c/general'

detections = []

# 단일 메세지 넣을 시
# if image:
total_len = len(image_files)
cnt = 1

for image in image_files: # 다량의 이미지 테스트 용
    original_img, cropped_img = predict2crop(model, folder_path = image_folder ,image_file = image)
    print(f'process : {(cnt/total_len*100) : .2f}%')
    cnt += 1

    if cropped_img is None : continue

    # cropped_img = cv2.resize(cropped_img,(128,64)) # 필수

    response = CLOVA_api(secret_key ,api_url ,image = cropped_img)
    texts, img = imageOCR(response, img = cropped_img)
    print(texts)


    detections.append(texts)


process :  0.64%
d77.jpeg
Detected :  2
! Error : Multiple detection Error. Take more closer
process :  1.27%
d66.jpeg
Detected :  0
SUCCESS
enty1.5~1.11 11,000[DIARY] 날짜형TWENTYTWENTY 8,800FOUR 2024 MINI
process :  1.91%
d73.jpeg
Detected :  1
SUCCESS
노지감귤국내산3KG/BOX₩35,000
process :  2.55%
d87.jpeg
Detected :  0
SUCCESS
먹태깡DR PEF RO 빼빼로CRUNKY
process :  3.18%
d95.jpeg
Detected :  2
! Error : Multiple detection Error. Take more closer
process :  3.82%
d65.jpeg
Detected :  1
SUCCESS
AvandressST/NO :PV.23(WA)M0042STYLE :BALLOONDEPECORDANIAN2N17FCOLOR CHAOCOAI (18)8SIZE ICNO Vi4-7-5-3DRICE ₩55,000ST/NO:PV23FWKNO042·CHARCOAL (3) COLORSITE :5₩55,000 385 PRICE -
process :  4.46%
d67.jpeg
Detected :  1
SUCCESS
멜로골드자몽 미국산개₩8, ,000
process :  5.10%
d68.jpeg
Detected :  1
SUCCESS
불로초감귤 국내산3KG/BOX₩45 ,000
process :  5.73%
d74.jpeg
Detected :  1
SUCCESS
토마토 73원산지 구내산2024.01.06 1200 1188JINUUPHUNNS 2011.6179 10M 142560 56544 32356현대식품관 13###
process :  6.37%
d79.jpeg
Detected :  1
SUCCESS
나뚜루)바닐라파인트

In [1]:
cv2_imshow(original_img)
cv2_imshow(cropped_img)

NameError: name 'cv2_imshow' is not defined

In [18]:
file_path = '/content/detected.txt'

with open(file_path, 'w') as file :
    for s in detections:
        file.write(s + '\n')


In [ ]:
text = ''
for t in detections:
    if t == '': continue
    text += t
    text += '\n'
print(text)

NameError: name 'detections' is not defined

- Text save

## Text preprocessing

In [ ]:
' '.join(texts)

'일반냉장고 R-B432GCWP.AKOR LG전자 나노참숯탈취 크기(WxHxD) 멀티냉각방식 755x1777x707 특급냉장 Bar 핸들타입'

In [ ]:
for d in detections:
    print(' '.join(d))

adidas SOLAR RIDE M CE449
주간추천상품 신라면멀티팩120g*5 2,490원
성동명 (원전시) 포장 년,월,일 두층 년,월,일 100g(원)중량(g) 100 10 가격(원) 1000 BARCORE DTHER
3238 553568 123 10 1/2 1285cvml ### 28S ### DEVALUAL 86585558 ### 400 Mid NO
삼성 갤럭시 ZFLIP SM-F700NZPAAKOO (0) 자급제폰 1,650,000 1,649,980 기준요금제: 자급제폰 24개월기준 출고가 통신사 지원금 아이마트 지원금 1,650,000 10 10 화면크기 용량 RAM 배터리 256GB (대각선) 12GB 3300mAh 170.2mm
LACARE 610 87 .........
쿠키 사이드 테이블 42 xH870 030 W1196 ₩49,900
푸르밀 비타요구르트 65ml*20입 806371 340163 판매가: 2,700원
빙그레요구르트 65ml+20의 07 672092 판매가: 3,000원
동원/요쿠르트 65ml*5*3 801155 403225 판매가: 3,000원
바이오플레-플레인 150ml*8입 판매가 : 3,400원
2014코카콜라브라질 250ml 1,250 8801094623203 /E 코카콜라음료
동서)맥심모카라이트 20T 801037 067651 판매가: 4,400원
맥심슈프림골드 20입 01037 00304 판매가: 4,900원
맥스웰커피믹스 240g 80103 7 035650 판매가: 2,500원
빙)요플레복숭아 100g*4개 01104 240390 판매가: 3,750원
0 빙그레 기획 닥터캡슐사과 520ml / 10ml당 86원 4,500원
₩ 19,000 SIZE EUR 20/21 자율안전확인신고필증번호 BO44R1466-1170 제조년월: 16.12 03179354904 061 6561/1 H&M KR/S11 0461949 001
켈로그콘푸로스트 600g 801083001425 판매가: 3,950원
유기농 유기농 인증 유기농원